# Participant ID:S02

### Content:
- Correlation matrix 
- Comfort Function implementation
- Weather Station Data (During Commuting)
- Weather Station Data (Day of Commuting from 8am to 6pm)
- Living Lab (Day of Commuting from 8am to 6pm)

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from datetime import datetime


In [ ]:
## Read the CSV file into a DataFrame
S02_path = 'C:/Users/Tomar/dev/PCM_study/summer_2023/output/process_data/S02_final_df.csv'

S02_df = pd.read_csv(S02_path, index_col=False, parse_dates=['DateTime'])

In [ ]:
S02_df.columns

In [ ]:
# Set 'DateTime' column as the index
S02_df.set_index('DateTime', inplace=True)

# Group by the unique ID instances
grouped = S02_df.groupby('ID_instance')

# Prepare a list to store the results
results = []

# Iterate through each group
for name, group in grouped:
    # Sort the group by DateTime if not already sorted
    group = group.sort_index()
    
    # Calculate the duration
    start_time = group.index[0]
    end_time = group.index[-1]
    duration = end_time - start_time
    
    # Calculate the min, max, and avg for Temp(C)_N1 and RH(%)_N1
    temp_n1_min = group['Temp(C)_N1'].min()
    temp_n1_max = group['Temp(C)_N1'].max()
    temp_n1_avg = group['Temp(C)_N1'].mean()
    
    rh_n1_min = group['RH(%)_N1'].min()
    rh_n1_max = group['RH(%)_N1'].max()
    rh_n1_avg = group['RH(%)_N1'].mean()
    
    # Append the result to the list
    results.append([name, start_time, end_time, duration, 
                    temp_n1_min, temp_n1_max, temp_n1_avg, 
                    rh_n1_min, rh_n1_max, rh_n1_avg])

# Create a DataFrame from the results list
results_df = pd.DataFrame(results, columns=['ID_instance', 'Start Time', 'End Time', 'Duration', 
                                            'Temp(C)_N1_min', 'Temp(C)_N1_max', 'Temp(C)_N1_avg',
                                            'RH(%)_N1_min', 'RH(%)_N1_max', 'RH(%)_N1_avg'])

# Print the results as a table
print(results_df)

In [ ]:
# Group by 'id_instance' and calculate time duration
time_duration = S02_df.groupby('ID_instance').apply(lambda x: x.index.max() - x.index.min())

# Convert time duration to seconds for better readability (optional)
time_duration = (time_duration.dt.total_seconds()/60).round(2)
# time_duration = time_duration.round(2)

print(time_duration)

In [ ]:
instances_over_60_minutes = time_duration[time_duration > 60].index

# Remove rows corresponding to instances over 60 minutes inplace
S02_df.drop(S02_df[S02_df['ID_instance'].isin(instances_over_60_minutes)].index, inplace=True)


In [ ]:
S02_df.isnull().sum()

In [ ]:
S02_df.dropna(subset=['ID_instance'], inplace=True)

In [ ]:
S02_df['ID_instance'].unique()

In [ ]:
S02_df.columns

In [ ]:
# Drop columns starting with 'acc_'
S02_df = S02_df.filter(regex='^(?!acc_)')

In [ ]:
S02_df.columns[1:18]

In [ ]:
S02_df.columns[18:30]

### Correlation matrix

In [ ]:
# Selecting physiological features (excluding Subject and Commuting columns)
physiological_data = S02_df.columns[1:18] 

# Selecting environmental parameters
environmental_data = S02_df.columns[18:30] 

# Calculating correlation matrix
correlation_matrix = S02_df[list(physiological_data) + list(environmental_data)].corr(method='spearman')

# Extracting correlations between physiological features and environmental parameters
correlation_phys_env = correlation_matrix.loc[physiological_data, environmental_data]

correlation_phys_env


### Correlation heatmap

In [ ]:
# Define the new column names with LaTeX subscript representation
new_column_names = {
    'Temp(C)_N1': 'T$_{1}$',
    'RH(%)_N1': 'H$_{1}$',
    'Dewpoint(C)_N1': 'D$_{1}$',
    'Temp(C)_N2': 'T$_{2}$',
    'RH(%)_N2': 'H$_{2}$',
    'Dewpoint(C)_N2': 'D$_{2}$',
    'Temp(C)_N3': 'T$_{3}$',
    'RH(%)_N3': 'H$_{3}$',
    'Dewpoint(C)_N3': 'D$_{3}$',
    'Temp(C)_N4': 'T$_{4}$',
    'RH(%)_N4': 'H$_{4}$',
    'Dewpoint(C)_N4': 'D$_{4}$',

    'hrv_vlf': 'HRV$_1$',
    'hrv_lf': 'HRV$_2$',
    'hrv_hf': 'HRV$_3$',
    'hrv_sdnn': 'HRV$_4$',
    'hrv_rmssd': 'HRV$_5$',
    'hrv_mean_nni': 'HRV$_6$',
    'hrv_median_nni': 'HRV$_7$',
    'hrv_range_nni': 'HRV$_8$',
    'hrv_entropy': 'HRV$_9$',

    'eda_tonic_mean': 'EDA$_1$',
    'eda_tonic_std': 'EDA$_2$',
    'eda_tonic_sum': 'EDA$_3$',
    'eda_tonic_energy': 'EDA$_4$',
    'eda_phasic_mean': 'EDA$_5$',
    'eda_phasic_std': 'EDA$_6$',
    'eda_phasic_sum': 'EDA$_7$',
    'eda_phasic_energy': 'EDA$_8$',

    # 'acc_acc_x_min': 'ACC$_1$',
    # 'acc_acc_y_min': 'ACC$_2$' ,
    # 'acc_acc_z_min': 'ACC$_3$', 
    # 'acc_l2_min': 'ACC$_4$',
    # 'acc_acc_x_max': 'ACC$_5$',
    # 'acc_acc_y_max': 'ACC$_6$', 
    # 'acc_acc_z_max': 'ACC$_7$', 
    # 'acc_l2_max': 'ACC$_8$',
    # 'acc_acc_x_ptp': 'ACC$_9$', 
    # 'acc_acc_y_ptp': 'ACC$_{10}$', 
    # 'acc_acc_z_ptp': 'ACC$_{11}$', 
    # 'acc_l2_ptp': 'ACC$_{12}$'
}

# Rename columns
correlation_phys_env_renamed = correlation_phys_env.rename(columns=new_column_names, index=new_column_names)

# Set up the matplotlib figure
plt.figure(figsize=(10, 6))

# Generate a heatmap of the correlation matrix
sns.heatmap(correlation_phys_env_renamed, annot=True, cmap='coolwarm', fmt=".2f")

# Add titles and labels
# plt.title('S02')
# plt.xlabel('environmental parameters', fontsize=14)
# plt.ylabel('physiological features', fontsize=14)

# Rotate the y-axis labels for better readability
plt.yticks(rotation=0)
plt.xticks(rotation=0)

# Save the plot as eps & png file
outpath = 'C:/Users/Tomar/dev/PCM_study/summer_2023/output/viz/'
plt.savefig(outpath+'S02_heatmap.eps', format='eps', bbox_inches='tight')
plt.savefig(outpath+'S02_heatmap.png', format='png', bbox_inches='tight')

# Show the plot
plt.show()


### Comfort function Implementation

In [ ]:
# List of all physiological variables
physio_vars = ['hrv_vlf', 'hrv_lf', 'hrv_hf', 'hrv_sdnn', 'hrv_rmssd',
               'hrv_mean_nni', 'hrv_median_nni', 'hrv_range_nni', 'hrv_entropy',
               'eda_tonic_mean', 'eda_tonic_std', 'eda_tonic_sum', 'eda_tonic_energy',
               'eda_phasic_mean', 'eda_phasic_std', 'eda_phasic_sum', 'eda_phasic_energy']

# Environmental variables to consider
env_vars = ['Temp(C)_N1', 'RH(%)_N1']

# Compute Spearman correlations
correlations = S02_df[physio_vars + env_vars].corr(method='spearman')

# HRV and EDA groups for correlation
hrv_vars = physio_vars[:9]  # Assuming the first 9 are HRV related
eda_vars = physio_vars[9:]  # Assuming the rest are EDA related

# Identify one HRV and one EDA variable with the strongest correlation with any environmental variable
strongest_corr_hrv = correlations.loc[hrv_vars, env_vars].abs().max().idxmax()
strongest_corr_eda = correlations.loc[eda_vars, env_vars].abs().max().idxmax()

# Extract the specific HRV and EDA variable names with the highest correlation
strongest_corr_hrv_var = correlations.loc[hrv_vars, env_vars].abs().idxmax()[strongest_corr_hrv]
strongest_corr_eda_var = correlations.loc[eda_vars, env_vars].abs().idxmax()[strongest_corr_eda]

# Define thresholds using the identified variables and adjusted quartiles for environmental variables
thresholds = {
    strongest_corr_hrv_var: S02_df[strongest_corr_hrv_var].quantile(0.75),
    strongest_corr_eda_var: S02_df[strongest_corr_eda_var].quantile(0.25),
    'Temp(C)_N1': [S02_df['Temp(C)_N1'].quantile(0.30), S02_df['Temp(C)_N1'].quantile(0.65)],
    'RH(%)_N1': [S02_df['RH(%)_N1'].quantile(0.30), S02_df['RH(%)_N1'].quantile(0.65)]
}

# Define the comfort function
def comfort_label(row):
    comfort = (row[strongest_corr_hrv_var] >= thresholds[strongest_corr_hrv_var] and
               row[strongest_corr_eda_var] <= thresholds[strongest_corr_eda_var] and
               thresholds['Temp(C)_N1'][0] <= row['Temp(C)_N1'] <= thresholds['Temp(C)_N1'][1] and
               thresholds['RH(%)_N1'][0] <= row['RH(%)_N1'] <= thresholds['RH(%)_N1'][1])
    
    if comfort:
        return 1
    
    discomfort = (row[strongest_corr_hrv_var] < thresholds[strongest_corr_hrv_var] and
                  row[strongest_corr_eda_var] > thresholds[strongest_corr_eda_var] and
                  not (thresholds['Temp(C)_N1'][0] <= row['Temp(C)_N1'] <= thresholds['Temp(C)_N1'][1]) or
                  not (thresholds['RH(%)_N1'][0] <= row['RH(%)_N1'] <= thresholds['RH(%)_N1'][1]))
    
    if discomfort:
        return 0
    
    return 0.5  # Sensible label for ambiguous cases

# Apply the comfort function and count labels
S02_df['lambda'] = S02_df.apply(comfort_label, axis=1)
label_counts = S02_df['lambda'].value_counts(normalize=True) * 100

# Print the names of variables used for the comfort function
print("Variables considered for the comfort function:")
print(f"HRV Variable: {strongest_corr_hrv_var}")
print(f"EDA Variable: {strongest_corr_eda_var}")
print("Environmental Variables: Temp(C)_N1, RH(%)_N1")

# Print the percentage of labeled values
print("Label Distribution (%):")
print(label_counts)

print("saving data files as pkl")
savepath = 'C:/Users/Tomar/dev/PCM_study/summer_2023/output/final_data/data_S02.pkl'
S02_df.to_pickle(savepath)

In [ ]:
# Group by unique instances
instance_groups = S02_df.groupby('ID_instance')

# Calculate the level of comfort for each instance
comfort_levels = instance_groups['lambda'].mean()  # You can also use .median() or .mode() depending on your preference

# Print the comfort levels
print(comfort_levels)


### Weather Station Data (During Commuting):

In [ ]:
import pandas as pd

# Load the weather data from the uploaded Excel file
file_path = 'C:/Users/Tomar/dev/datasets/weather_summer2023.xlsx'
weather_data = pd.read_excel(file_path)

# Display the first few rows of the weather data to understand its structure
weather_data.head()


In [ ]:
# Define the commuting instance data
commuting_data = [
    {"ID_instance": "S02_1", "Start Time" : "2023-07-31 16:33:54", "End Time": "2023-07-31 16:36:38"},
    {"ID_instance": "S02_2", "Start Time" : "2023-08-01 16:38:07", "End Time": "2023-08-01 16:54:32"},
    {"ID_instance": "S02_3", "Start Time" : "2023-08-01 20:34:40", "End Time": "2023-08-01 20:45:29"},
    {"ID_instance": "S02_4", "Start Time" : "2023-08-02 07:13:16", "End Time": "2023-08-02 07:26:44"},
    {"ID_instance": "S02_5", "Start Time" : "2023-08-02 11:06:22", "End Time": "2023-08-02 11:16:42"},
    {"ID_instance": "S02_6", "Start Time" : "2023-08-02 12:33:00", "End Time": "2023-08-02 12:41:28"},
    {"ID_instance": "S02_7", "Start Time" : "2023-08-02 16:18:34", "End Time": "2023-08-02 16:28:48"},
    {"ID_instance": "S02_8", "Start Time" : "2023-08-03 07:09:01", "End Time": "2023-08-03 07:18:02"},
    {"ID_instance": "S02_9", "Start Time" : "2023-08-03 16:39:40", "End Time": "2023-08-03 16:46:27"},
    {"ID_instance": "S02_10", "Start Time": "2023-08-03 17:21:02", "End Time": "2023-08-03 17:27:24"},
    {"ID_instance": "S02_11", "Start Time": "2023-08-04 22:13:08", "End Time": "2023-08-04 22:24:06"},
    {"ID_instance": "S02_12", "Start Time": "2023-08-05 05:49:36", "End Time": "2023-08-05 06:29:37"},
    {"ID_instance": "S02_13", "Start Time": "2023-08-05 14:05:21", "End Time": "2023-08-05 14:11:50"},
    {"ID_instance": "S02_14", "Start Time": "2023-08-05 15:47:28", "End Time": "2023-08-05 16:17:37"},
    {"ID_instance": "S02_15", "Start Time": "2023-08-06 08:48:20", "End Time": "2023-08-06 09:31:08"},
    {"ID_instance": "S02_16", "Start Time": "2023-08-06 09:48:21", "End Time": "2023-08-06 09:59:58"},
    {"ID_instance": "S02_17", "Start Time": "2023-08-07 07:07:35", "End Time": "2023-08-07 07:15:30"},
    {"ID_instance": "S02_18", "Start Time": "2023-08-07 12:52:42", "End Time": "2023-08-07 12:59:34"},
    {"ID_instance": "S02_19", "Start Time": "2023-08-07 18:21:39", "End Time": "2023-08-07 18:27:22"},
    {"ID_instance": "S02_20", "Start Time": "2023-08-08 06:40:41", "End Time": "2023-08-08 06:50:55"},
    {"ID_instance": "S02_21", "Start Time": "2023-08-08 10:18:33", "End Time": "2023-08-08 10:25:43"},
    {"ID_instance": "S02_22", "Start Time": "2023-08-08 16:43:22", "End Time": "2023-08-08 16:53:31"},
    {"ID_instance": "S02_23", "Start Time": "2023-08-09 06:32:09", "End Time": "2023-08-09 06:39:00"},
    {"ID_instance": "S02_24", "Start Time": "2023-08-09 16:07:41", "End Time": "2023-08-09 16:14:22"},
    {"ID_instance": "S02_25", "Start Time": "2023-08-10 07:19:12", "End Time": "2023-08-10 07:26:47"},
    {"ID_instance": "S02_26", "Start Time": "2023-08-10 11:01:34", "End Time": "2023-08-10 11:12:30"},
    {"ID_instance": "S02_27", "Start Time": "2023-08-10 12:42:41", "End Time": "2023-08-10 12:48:16"},
    {"ID_instance": "S02_28", "Start Time": "2023-08-10 16:52:05", "End Time": "2023-08-10 17:01:36"},
    {"ID_instance": "S02_29", "Start Time": "2023-08-11 07:08:23", "End Time": "2023-08-11 07:15:03"},
    {"ID_instance": "S02_30", "Start Time": "2023-08-11 12:01:19", "End Time": "2023-08-11 12:07:27"},




]

# Convert the commuting instance data to a DataFrame
commuting_df = pd.DataFrame(commuting_data)

# Convert the Start Time and End Time columns to datetime
commuting_df['Start Time'] = pd.to_datetime(commuting_df['Start Time'])
commuting_df['End Time'] = pd.to_datetime(commuting_df['End Time'])

# Display the commuting instance DataFrame
commuting_df.head()


In [ ]:
# Convert the Data column in weather data to datetime
weather_data['Data'] = pd.to_datetime(weather_data['Data'])

# Initialize an empty list to store the results
results = []

# Iterate over each commuting instance
for _, row in commuting_df.iterrows():
    # Filter the weather data within the date-time range of the commuting instance
    mask = (weather_data['Data'] >= row['Start Time']) & (weather_data['Data'] <= row['End Time'])
    filtered_data = weather_data[mask]
    
    # Calculate the average, min, and max for Air Temperature and Relative Humidity 
    min_temp = filtered_data['Air Temperature Min (\'C)'].min()
    max_temp = filtered_data['Air Temperature Max (\'C)'].max()
    avg_temp = filtered_data['Air Temperature Ave (\'C)'].mean()

    min_rh = filtered_data['Relative Humidity Min (%)'].min()
    max_rh = filtered_data['Relative Humidity Max (%)'].max()
    avg_rh = filtered_data['Relative Humidity Ave (%)'].mean()
    
    # min_sr = filtered_data['Solar Radiation Ave (W/m2)'].min()
    # max_sr = filtered_data['Solar Radiation Ave (W/m2)'].max()
    avg_sr = filtered_data['Solar Radiation Ave (W/m2)'].mean()

    # Append the results
    results.append({
        'ID_instance': row['ID_instance'],
        'Min_Air_Temperature': min_temp,
        'Max_Air_Temperature': max_temp,
        'Avg_Air_Temperature': avg_temp,
        'Min_Relative_Humidity': min_rh,
        'Max_Relative_Humidity': max_rh,
        'Avg_Relative_Humidity': avg_rh,
        'Avg_Solar_Radiation': avg_sr
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results
results_df


### Weather Station Data (Day of Commuting from 8am to 6pm):

In [ ]:
import pandas as pd

# Load the data from the provided Excel file
file_path = 'C:/Users/Tomar/dev/datasets/weather_summer2023.xlsx'
data = pd.read_excel(file_path)

# Convert the 'Data' column to datetime format
data['Data'] = pd.to_datetime(data['Data'])

# Define the date ranges as given in the task
date_ranges = [
    ("2023-07-31 08:00:00", "2023-07-31 18:00:00"),
    ("2023-08-01 08:00:00", "2023-08-01 18:00:00"),
    ("2023-08-02 08:00:00", "2023-08-02 18:00:00"),
    ("2023-08-03 08:00:00", "2023-08-03 18:00:00"),
    ("2023-08-04 08:00:00", "2023-08-04 18:00:00"),
    ("2023-08-07 08:00:00", "2023-08-07 18:00:00"),
    ("2023-08-08 08:00:00", "2023-08-08 18:00:00"),
    ("2023-08-09 08:00:00", "2023-08-09 18:00:00"),
    ("2023-08-10 08:00:00", "2023-08-10 18:00:00"),
    ("2023-08-11 08:00:00", "2023-08-11 18:00:00")
]

# Initialize a list to store statistics for each day
daily_statistics = []

# Calculate statistics for each date range
for start, end in date_ranges:
    day_data = data[(data['Data'] >= start) & (data['Data'] <= end)]
    
    stats = {
        'Date Range': f'{start} to {end}',
        'Min_Air_Temperature': day_data["Air Temperature Min ('C)"].min(),
        'Max_Air_Temperature': day_data["Air Temperature Max ('C)"].max(),
        'Avg_Air_Temperature': day_data["Air Temperature Ave ('C)"].mean(),
        'Min_Relative_Humidity': day_data['Relative Humidity Min (%)'].min(),
        'Max_Relative_Humidity': day_data['Relative Humidity Max (%)'].max(),
        'Avg_Relative_Humidity': day_data['Relative Humidity Ave (%)'].mean(),
        'Avg_Solar_Radiation': day_data['Solar Radiation Ave (W/m2)'].mean()
    }
    
    daily_statistics.append(stats)

# Convert the results to a DataFrame for better readability
daily_statistics_df = pd.DataFrame(daily_statistics)
daily_statistics_df


### Living Lab (Day of Commuting from 8am to 6pm):

In [ ]:
# Load the CSV file
file_path = 'C:/Users/Tomar/dev/PCM_study/summer_2023/output/living_lab/LL/LL_b.csv'
data = pd.read_csv(file_path)

# Ensure the correct renaming of the 'DateTime' column to 'Timestamp'
data.rename(columns={'DateTime': 'Timestamp'}, inplace=True)

# Convert the 'Timestamp' column to datetime
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

# Define the start and end times for each day
time_intervals = [
    ("2023-07-31 08:00:00", "2023-07-31 18:00:00"),
    ("2023-08-01 08:00:00", "2023-08-01 18:00:00"),
    ("2023-08-02 08:00:00", "2023-08-02 18:00:00"),
    ("2023-08-03 08:00:00", "2023-08-03 18:00:00"),
    ("2023-08-04 08:00:00", "2023-08-04 18:00:00"),
    ("2023-08-07 08:00:00", "2023-08-07 18:00:00"),
    ("2023-08-08 08:00:00", "2023-08-08 18:00:00"),
    ("2023-08-09 08:00:00", "2023-08-09 18:00:00"),
    ("2023-08-10 08:00:00", "2023-08-10 18:00:00"),
    ("2023-08-11 08:00:00", "2023-08-11 18:00:00")
    
]

# Convert string times to datetime
time_intervals = [(datetime.strptime(start, "%Y-%m-%d %H:%M:%S"), datetime.strptime(end, "%Y-%m-%d %H:%M:%S")) for start, end in time_intervals]

# Initialize a list to store results
results_list = []

# Process each time interval
for start, end in time_intervals:
    # Filter data based on time interval
    filtered_data = data[(data['Timestamp'] >= start) & (data['Timestamp'] <= end)]
    
    # Calculate required metrics
    result = {
        'Start_Time': start,
        'End_Time': end,
        'Min_Temp_Air': filtered_data['Temp_Air(C)'].min(),
        'Max_Temp_Air': filtered_data['Temp_Air(C)'].max(),
        'Avg_Temp_Air': filtered_data['Temp_Air(C)'].mean(),
        'Min_Temp_Globe': filtered_data['Temp_Globe(C)'].min(),
        'Max_Temp_Globe': filtered_data['Temp_Globe(C)'].max(),
        'Avg_Temp_Globe': filtered_data['Temp_Globe(C)'].mean(),
        'Min_Relative_Humidity': filtered_data['RH(%)'].min(),
        'Max_Relative_Humidity': filtered_data['RH(%)'].max(),
        'Mean_Relative_Humidity': filtered_data['RH(%)'].mean(),
    }
    
    # Append result to the list
    results_list.append(result)

# Create a DataFrame from the results list
results_df = pd.DataFrame(results_list)

# Display the results
results_df
